<a href="https://colab.research.google.com/github/thdusqpf/verlan/blob/main/verlantoenglish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare special letter

In [2]:
space_letter = ' '
special_letters = [space_letter]

# Prepare English data

In [3]:
first_big_id = ord('A')
last_big_id = ord('Z')
first_small_id = ord('a')
last_small_id = ord('z')

big = list(range(first_big_id, last_big_id + 1))
small = list(range(first_small_id, last_small_id + 1))

english_letter_list = [chr(num) for num in big + small]

english_letter_list.extend(special_letters)

idx2english = dict(enumerate(english_letter_list, 1)) #1부터 시작
print(idx2english.items())
english2idx = {v: k for k, v in idx2english.items()}


dict_items([(1, 'A'), (2, 'B'), (3, 'C'), (4, 'D'), (5, 'E'), (6, 'F'), (7, 'G'), (8, 'H'), (9, 'I'), (10, 'J'), (11, 'K'), (12, 'L'), (13, 'M'), (14, 'N'), (15, 'O'), (16, 'P'), (17, 'Q'), (18, 'R'), (19, 'S'), (20, 'T'), (21, 'U'), (22, 'V'), (23, 'W'), (24, 'X'), (25, 'Y'), (26, 'Z'), (27, 'a'), (28, 'b'), (29, 'c'), (30, 'd'), (31, 'e'), (32, 'f'), (33, 'g'), (34, 'h'), (35, 'i'), (36, 'j'), (37, 'k'), (38, 'l'), (39, 'm'), (40, 'n'), (41, 'o'), (42, 'p'), (43, 'q'), (44, 'r'), (45, 's'), (46, 't'), (47, 'u'), (48, 'v'), (49, 'w'), (50, 'x'), (51, 'y'), (52, 'z'), (53, ' ')])


# xlsx to csv


In [4]:
xlsx = pd.read_excel('verlandataset.xlsx')
xlsx.to_csv('verlandataset.csv')

# Load dataset from csv

In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('verlandataset.csv', encoding='utf-8')
del df['Unnamed: 0']
df.columns

Index(['verlan', 'original'], dtype='object')

In [13]:
#Load X
verlan = df['verlan'].apply(lambda x: np.array([english2idx[letter] for letter in x]))

verlan_df = pd.DataFrame(verlan)

X = pd.DataFrame(verlan_df['verlan'].tolist()).values

#Replace nan to 0
X[np.isnan(X)] = 0
print(X)


[[ 3. 27. 35. ...  0.  0.  0.]
 [ 2. 31. 47. ...  0.  0.  0.]
 [18. 31. 28. ...  0.  0.  0.]
 ...
 [13. 31. 47. ...  0.  0.  0.]
 [19. 31. 47. ...  0.  0.  0.]
 [20. 31. 47. ...  0.  0.  0.]]


In [14]:
X.shape

(99, 23)

In [ ]:
#Flip the input
X = np.flip(X, axis = 1)
X

array([[ 0.,  0.,  0., ..., 35., 27.,  3.],
       [ 0.,  0.,  0., ..., 47., 31.,  2.],
       [ 0.,  0.,  0., ..., 28., 31., 18.],
       ...,
       [ 0.,  0.,  0., ..., 47., 31., 13.],
       [ 0.,  0.,  0., ..., 47., 31., 19.],
       [ 0.,  0.,  0., ..., 47., 31., 20.]])

In [2]:
#Split train & test & validation set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.3)

NameError: name 'X' is not defined

In [ ]:
#Get original X
original_X_test = np.flip(x_test, axis = 1)
original_X_test = [''.join([idx2english[j]for j in i if j != 0]) for i in original_X_test]

#RNN
- 순서가 있는 시퀀스 데이터를 입력받아 순서대로 하나씩 데이터 처리
- 모든 입력 데이터를 한번에 처리하는 MLP, CNN 신경망과 차이점 존재
- 다음 단어(정답)와 출력(예측)의 차이(cost function)를 최소화하는 가중치를 학습함.
-레이어를 여러 개 거치면서 처음에 입력했던 단어에 대한 정보를 조금씩 잃어버림 -> 최근 학습 단어가 모델의 최종 예측에 더 큰 영향을 줌.

##return_sequences 옵션
- True: 모든 시간의 출력값을 그대로 다음 레이어에 전달
- False or 지정 x: 최종 레이어의 출력만 반환

#LSTM
- RNN의 단점 보완 모델(오래 전에 학습한 데이터 기억 못함)
- 기존 정보 중에서 중요한 정보를 다음 단계에 전달하는 구조 도입(장기 기억 성능 개선)
- 길이 긴 시퀀스 데이터 학습에 적합


#GRU
- LSTM의 느린 학습 속도(시퀀스의 길이 길어지면 학습해야 할 가중치 파라미터의 개수가 기하급수적으로 증가) 개선 위해 제안된 알고리즘
- LSTM의 셀을 단순한 구조로 변경 -> 모델의 파라미터 개수 줄이고 학습 속도 개선한 모델



#Seq2Seq
참고 블로그:
https://velog.io/@bobaebak/NLP-sequence-to-sequence%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%B4%EC%84%9C-%EB%B2%88%EC%97%AD%EA%B8%B0-%EB%A7%8C%EB%93%A4%EA%B8%B0

In [1]:
!pip install recurrent
from keras.layers import recurrent

encoder_RNN = recurrent.LSTM
decoder_RNN = recurrent.GRU
stop_monitor = 'val_acc'
stop_delta = 0.0
stop_epochs = 20

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for recurrent: filename=recurrent-0.4.1-py3-none-any.whl size=22213 sha256=8ecf450b056c20aa798a777f46eb4be1a2c1bf8ec1679cce00ad9558b4fee73a
  Stored in directory: /root/.cache/pip/wheels/5f/82/16/088f92476d450f3c52437c02ef91b61db47f332a54fef3c1d8
Successfully built recurrent


ImportError: cannot import name 'recurrent' from 'keras.layers' (/usr/local/lib/python3.10/dist-packages/keras/layers/__init__.py)


#keras Sequential API
- 레이어 여러 개를 연결하여 신경망 모델을 구성하는 도구

- 간단한 아키텍처를 가지면서도 대부분의 딥러닝 모델을 만들 수 있음.

##레이어
- 임베딩(embedding): 말뭉치(corpus)를 입력하여 단어 사이의 유사도와 여러 가지 관계를 벡터로 표현 가능

##용어
- 입력 데이터의 차원(input_dim): 모델 학습에 사용하는 설명 변수(피처)의 개수 지정

    ex)Y = aX + b에서 Y를 목표 변수(target), 목표 변수를 예측하는데 사용되는 X를 설명 변수(feature)라고 한다.
- output_dim: 임베딩으로 변환되는 출력 벡터의 차원

##summary 메서드
- 모델 아키텍처(구조) 확인하는 메서드

In [15]:
EMBEDDING_OUTPUT_SIZE = 200

# ## Build Model
from keras.models import Sequential, load_model
from keras.layers import Dense, TimeDistributed, Activation, RepeatVector, Embedding

print('Build Model...')

model = Sequential()

model.add(Embedding(
    input_dim= len(english_letter_list) + 1,
    output_dim=EMBEDDING_OUTPUT_SIZE,
    input_length=X.shape[1]
))

model.summary()

Build Model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 200)           10800     
                                                                 
Total params: 10800 (42.19 KB)
Trainable params: 10800 (42.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.add(encoder_RNN(HIDDEN_SIZE, return_sequences=True, input_shape=(X.shape[1], )))
model.add(encoder_RNN(HIDDEN_SIZE))

model.add(RepeatVector(y.shape[1]))  # Maximum output length

for _ in range(LAYERS):
    model.add(decoder_RNN(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributed(Dense(len(korean_letter_list)+1)))
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


model.summary()
print('Done')

#제로 패딩
- 딥러닝 모델에 입력하기 위해 입력 데이터의 크기 동일하게 조정

    왜 동일해야 하나? 병렬 연산이 어렵기 때문에
- 입력의 최대 길이(maxlen) 설정 후, 이보다 길이가 긴 경우 중간에 잘라서 길이를 맞추고, 짧은 경우에는 부족한만큼 숫자 0으로 채움.

##패딩 방향 설정
- default는 padding='pre'라서 앞에서부터 0을 채움.
-padding='post' 뒤에서부터 0 채움.

In [ ]:
from keras.models import load_model
model_name_to_read = './models/GRU2GRU_consonant-vowel_LSTM_flip_checkpoint'
loaded_model = load_model(model_name_to_read)

custom_input = ['crong', 'krong', 'hello', 'derek', 'rob', 'dennis']
from keras.preprocessing.sequence import pad_sequences
custom_X = pad_sequences([[english2idx[letter] for letter in word] for word in custom_input], maxlen=X.shape[1], padding='post')

custom_X = np.flip(custom_X, axis=1)

custom_results = loaded_model.predict(custom_X)
custom_argmax_result = np.argmax(custom_results, axis=2)
custom_readable_results = [''.join([idx2korean[let] for let in one_result if let != 0]) for one_result in custom_argmax_result]